In [8]:
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile
import os
import pandas as pd
import ast

1. Get your kaggle.json file:
- Go to https://www.kaggle.com/account where account is ur account name
- Scroll to the API section
- Click Create New API Token
- This downloads kaggle.json to your computer (usually in Downloads folder)


2. Put kaggle.json in the right folder
- Move the file to this folder:
    C:\Users\Morad Elshorbagy\\.kaggle\

If the .kaggle folder doesn’t exist, create it manually

In [9]:
# === STEP 0: Setup Kaggle API and download dataset ZIP ===
api = KaggleApi()
api.authenticate()

dataset_zip = 'the-movies-dataset.zip'
output_folder = 'Data'

# Download the dataset ZIP only if not already downloaded
if not os.path.exists(dataset_zip):
    print("Downloading dataset ZIP...")
    api.dataset_download_files('rounakbanik/the-movies-dataset', path='.', unzip=False)
else:
    print("Dataset ZIP already downloaded.")

Dataset ZIP already downloaded.


In [10]:
# Create data folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# === STEP 1: Extract all necessary files (only if missing) ===
needed_files = ['ratings.csv', 'movies_metadata.csv', 'credits.csv', 'keywords.csv']
existing_files = os.listdir(output_folder)

with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    for file in needed_files:
        if file not in existing_files:
            print(f"Extracting {file}...")
            zip_ref.extract(file, path=output_folder)
        else:
            print(f"{file} already extracted.")

print("Extraction complete.\n")

ratings.csv already extracted.
movies_metadata.csv already extracted.
Extracting credits.csv...
Extracting keywords.csv...
Extraction complete.



In [11]:
# === STEP 2: Load datasets ===
print("Loading datasets...")
movies = pd.read_csv(os.path.join(output_folder, 'movies_metadata.csv'), low_memory=False)
ratings = pd.read_csv(os.path.join(output_folder, 'ratings.csv'))
credits = pd.read_csv(os.path.join(output_folder, 'credits.csv'))
keywords = pd.read_csv(os.path.join(output_folder, 'keywords.csv'))
print("Datasets loaded.\n")

Loading datasets...
Datasets loaded.



In [20]:
# Inspect structure of all datasets
print("movies_metadata.csv columns:\n", movies.columns)
print("ratings.csv columns:\n", ratings.columns)
print("credits.csv columns:\n", credits.columns)
print("keywords.csv columns:\n", keywords.columns)

movies_metadata.csv columns:
 Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')
ratings.csv columns:
 Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')
credits.csv columns:
 Index(['cast', 'crew', 'id'], dtype='object')
keywords.csv columns:
 Index(['id', 'keywords'], dtype='object')


In [12]:
# === STEP 3: Make copies to keep originals intact ===
movies_clean = movies.copy()
ratings_clean = ratings.copy()
credits_clean = credits.copy()
keywords_clean = keywords.copy()

In [13]:
# === STEP 4: Define helper functions ===
def parse_json_column(json_str):
    """Parse JSON-like string to list of names."""
    try:
        items = ast.literal_eval(json_str)
        return [item['name'] for item in items]
    except (ValueError, SyntaxError):
        return []

def safe_int_conversion(val):
    """Safely convert to int, return None if fails."""
    try:
        return int(val)
    except:
        return None

In [16]:
# === STEP 5: Clean movies metadata ===
print("Cleaning movies metadata...")
# Convert budget and revenue to numeric, fill NaN with 0
movies_clean['budget'] = pd.to_numeric(movies_clean['budget'], errors='coerce').fillna(0)
movies_clean['revenue'] = pd.to_numeric(movies_clean['revenue'], errors='coerce').fillna(0)

# Drop rows missing critical info
movies_clean = movies_clean.dropna(subset=['title', 'id'])

# Convert 'id' to numeric and drop invalid rows
movies_clean['id'] = pd.to_numeric(movies_clean['id'], errors='coerce')
movies_clean = movies_clean.dropna(subset=['id'])
movies_clean['id'] = movies_clean['id'].astype(int)

# Parse genres column (JSON string) into list of genre names
movies_clean['genres'] = movies_clean['genres'].apply(parse_json_column)
# Replace empty genres lists with ['Unknown']
movies_clean['genres'] = movies_clean['genres'].apply(lambda x: x if x else ['Unknown'])
print("Movies metadata cleaned.\n")

# === STEP 6: Clean ratings data ===
print("Cleaning ratings data...")
# Drop duplicates (same userId, movieId)
ratings_clean = ratings_clean.drop_duplicates(subset=['userId', 'movieId'])
# Drop rows with missing essential columns
ratings_clean = ratings_clean.dropna(subset=['userId', 'movieId', 'rating'])
# Convert types properly
ratings_clean['userId'] = ratings_clean['userId'].astype(int)
ratings_clean['movieId'] = ratings_clean['movieId'].astype(int)
ratings_clean['rating'] = ratings_clean['rating'].astype(float)
print("Ratings data cleaned.\n")
# === STEP 7: Clean credits data ===
print("Cleaning credits data...")
credits_clean['cast'] = credits_clean['cast'].apply(parse_json_column)
credits_clean['crew'] = credits_clean['crew'].apply(parse_json_column)

credits_clean['id'] = credits_clean['id'].apply(safe_int_conversion)
credits_clean = credits_clean.dropna(subset=['id'])
credits_clean['id'] = credits_clean['id'].astype(int)
print("Credits data cleaned.\n")

# === STEP 8: Clean keywords data ===
print("Cleaning keywords data...")
keywords_clean['keywords'] = keywords_clean['keywords'].apply(parse_json_column)

keywords_clean['id'] = keywords_clean['id'].apply(safe_int_conversion)
keywords_clean = keywords_clean.dropna(subset=['id'])
keywords_clean['id'] = keywords_clean['id'].astype(int)
print("Keywords data cleaned.\n")

Cleaning movies metadata...
Movies metadata cleaned.

Cleaning ratings data...
Ratings data cleaned.

Cleaning credits data...
Credits data cleaned.

Cleaning keywords data...
Keywords data cleaned.



In [17]:
# === STEP 9: Merge datasets ===
print("Merging datasets...")

# Merge movies with credits (cast & crew)
movies_full = pd.merge(movies_clean, credits_clean[['id', 'cast', 'crew']], on='id', how='left')

# Merge movies_full with keywords
movies_full = pd.merge(movies_full, keywords_clean[['id', 'keywords']], on='id', how='left')

# Merge ratings with movies (to get movie metadata for ratings)
ratings_movies = pd.merge(ratings_clean, movies_clean, left_on='movieId', right_on='id', how='inner')

print("Datasets merged.\n")

Merging datasets...
Datasets merged.



In [18]:
# === STEP 10: Summary info ===
print(f"Movies dataset shape: {movies_clean.shape}")
print(f"Ratings dataset shape: {ratings_clean.shape}")
print(f"Credits dataset shape: {credits_clean.shape}")
print(f"Keywords dataset shape: {keywords_clean.shape}")
print(f"Movies full dataset (with credits & keywords) shape: {movies_full.shape}")
print(f"Ratings merged with movies shape: {ratings_movies.shape}\n")

Movies dataset shape: (45460, 24)
Ratings dataset shape: (26024289, 4)
Credits dataset shape: (45476, 3)
Keywords dataset shape: (46419, 2)
Movies full dataset (with credits & keywords) shape: (46625, 27)
Ratings merged with movies shape: (11437637, 28)



In [19]:
# === STEP 11: Save cleaned data to CSV for reuse ===
movies_full.to_csv(os.path.join(output_folder, 'movies_full_clean.csv'), index=False)
ratings_movies.to_csv(os.path.join(output_folder, 'ratings_movies_clean.csv'), index=False)

print(f"Cleaned datasets saved in '{output_folder}' folder.")

Cleaned datasets saved in 'Data' folder.
